In [ ]:
import serial
import pandas as pd
import matplotlib.pyplot as plt
import time
import datetime
import threading
import ipywidgets as widgets
from IPython.display import display

data_lock = threading.Lock()
ser = serial.Serial(port='COM3', baudrate=9600, bytesize=serial.SEVENBITS, parity=serial.PARITY_ODD, stopbits=serial.STOPBITS_ONE, timeout=1)
datapoints = []
timestamps = []
collecting = False
save_counter = 0
autosave_counter = 0

def collect_data():
    global collecting
    while collecting: 
        ser.write(bytes("KRDG?\r\n", 'utf-8'))
        response = ser.readline().decode('utf-8').strip()
        timestamp = datetime.datetime.now()
        timestamps.append(timestamp)
        datapoints.append(response)
        time.sleep(1)

def start_data_collection(b):
    global collecting
    if not collecting:
        collecting = True
        thread = threading.Thread(target=collect_data)
        thread.start()
        print("Data collection started.")
        schedule_autosave()

def stop_data_collection(b):
    global collecting
    collecting = False
    if autosave_timer:
        autosave_timer.cancel()
    ser.close()
    print("Data collection stopped.")

def save_data(b):
    global save_counter
    save_counter += 1
    filename = f'cooldown_curve_{save_counter}.csv'
    df = pd.DataFrame({
        'Timestamp': timestamps, 
        'Time': range(len(timestamps)), 
        'Temperature': datapoints
    })
    df['Temperature'] = pd.to_numeric(df['Temperature'], errors='coerce')
    df.to_csv(filename, index=False)
    print(f"Data saved to '{filename}'.")


def autosave_data():
    global autosave_counter, autosave_timer
    with data_lock:
        local_timestamps = timestamps.copy()
        local_datapoints = datapoints.copy()

    min_length = min(len(local_timestamps), len(local_datapoints))
    local_timestamps = local_timestamps[:min_length]
    local_datapoints = local_datapoints[:min_length]

    autosave_counter += 1
    filename = f'autosave/curve_{autosave_counter}.csv'
    df = pd.DataFrame({
        'Timestamp': local_timestamps, 
        'Time': range(len(local_timestamps)), 
        'Temperature': local_datapoints
    })
    df['Temperature'] = pd.to_numeric(df['Temperature'], errors='coerce')
    df.to_csv(filename, index=False)
    print(f"Data autosaved to '{filename}'.")

    schedule_autosave()


def schedule_autosave():
    global autosave_timer
    autosave_timer = threading.Timer(600, autosave_data)  # 600 seconds = 10 minutes
    autosave_timer.start()

def show_data(b):
    if datapoints:
        print(f"Current Temperature: {datapoints[-1]} K at {timestamps[-1]}")

    if datapoints:
        df = pd.DataFrame({'Temperature': datapoints, 'Timestamp': timestamps})
        df['Temperature'] = pd.to_numeric(df['Temperature'], errors='coerce')
        df['Time'] = range(len(timestamps))

        plt.rc('xtick', labelsize=16) 
        plt.rc('ytick', labelsize=16) 
        plt.rc('axes', labelsize=20)
        plt.rc('axes', titlesize=20)

        fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(18, 5))

        ax1.plot(df.Time / 3600, df.Temperature, color='r')
        ax1.set_xlabel('Time [h]')
        ax1.set_ylabel('Temperature [K]')
        ax1.set_xlim(0, 10)
        ax1.set_title('Cu-110 In-Situ Cooldown and Warmup')

        ax2.plot(df.Time / 60, df.Temperature, color='r')
        ax2.set_xlabel('Time [min]')
        ax2.set_ylabel('Temperature [K]')
        ax2.set_title('Cu-110 In-Situ Cooldown and Warmup')

        plt.tight_layout()
        plt.show()

start_button = widgets.Button(description="Start")
stop_button = widgets.Button(description="Stop")
save_button = widgets.Button(description="Save")
show_button = widgets.Button(description="Show")

start_button.on_click(start_data_collection)
stop_button.on_click(stop_data_collection)
save_button.on_click(save_data)
show_button.on_click(show_data)

display(start_button, stop_button, save_button, show_button)